<a href="https://colab.research.google.com/github/Deepthi-Nadar/Movie-recommendations-system-using-ml/blob/main/MovieRecommendatingSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Data collection and pre-processing**

In [ ]:
#loading the dataset
df=pd.read_csv('/content/movies.csv')

In [ ]:
#printing 5 rows of the data
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#number of rows and columns
df.shape

(4803, 24)

In [ ]:
#Feature selection (selecting the relevant features for recommendations)
selected_features = ['genres','keywords', 'tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
#replacing the null values with the null strong

for feature in selected_features:
    df[feature] = df[feature].fillna('')

In [ ]:
print(df)

      index     budget                                    genres  \
0         0  237000000  Action Adventure Fantasy Science Fiction   
1         1  300000000                  Adventure Fantasy Action   
2         2  245000000                    Action Adventure Crime   
3         3  250000000               Action Crime Drama Thriller   
4         4  260000000          Action Adventure Science Fiction   
...     ...        ...                                       ...   
4798   4798     220000                     Action Crime Thriller   
4799   4799       9000                            Comedy Romance   
4800   4800          0             Comedy Drama Romance TV Movie   
4801   4801          0                                             
4802   4802          0                               Documentary   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.go.com/disneypictures/pi

In [ ]:
#combining all the selected features

combined_features = df['genres']+' ' +df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
#converting the text data to feature vectors

vectorizer=TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

**Cosine Similarity**

In [98]:
#getting the similarity scores using cosine Similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)
similarity.shape

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


(4803, 4803)

In [84]:
#getting the movie name from the user

movie_name=input("Enter your favourite movie name: ")

Enter your favourite movie name: The nun 


In [85]:
#creating a list with all the movie names given in the dataset

list_of_all_titles=df['title'].tolist()

In [87]:
#finding the close match for the movie name given by the user

close_match=difflib.get_close_matches(movie_name, list_of_all_titles)
print(close_match)

['The Hunt', 'The Ruins', 'The Queen']


In [88]:
close=close_match[0]
print(close)

The Hunt


In [93]:
#finding the index of the movie with title


index_of_the_movie=df[df.title==close]['index'].values[0]
print(index_of_the_movie)

3870


In [94]:
#getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [95]:
len(similarity_score)

4803

In [99]:
#sorting the movies based on their similarity scores

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True)

In [97]:
#print the name of similar movies based on the index

print('Movies Suggestion for you:  \n')

i=1

for movie in sorted_similar_movies:
    index =movie[0]
    title_from_index = df[df.index==index]['title'].values[0]
    if (i<11):

        print(i,':', title_from_index)
        i=i+1

Movies Suggestion for you:  

1 : The Hunt
2 : The Celebration
3 : The Adventures of Pinocchio
4 : Thank You for Smoking
5 : Beowulf
6 : Casino Royale
7 : Dear Wendy
8 : Flame & Citron
9 : Special
10 : He Got Game


In [ ]:
movie_name=input("Enter your favourite movie name: ")
list_of_all_titles=df['title'].tolist()

close_match=difflib.get_close_matches(movie_name, list_of_all_titles)
close=close_match[0]
index_of_the_movie=df[df.title==close]['index'].values[0]


similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True)

print('Movies Suggestion for you:  \n')

i=1

for movie in sorted_similar_movies:
    index =movie[0]
    title_from_index = df[df.index==index]['title'].values[0]
    if (i<11):

        print(i,':', title_from_index)
        i=i+1

Enter your favourite movie name: Iron man 
Movies Suggestion for you:  

1 : Iron Man 3
2 : Iron Man 2
3 : Iron Man
4 : Avengers: Age of Ultron
5 : Captain America: Civil War
6 : The Avengers
7 : Ant-Man
8 : X-Men: Days of Future Past
9 : Kiss Kiss Bang Bang
10 : The Helix... Loaded
